In [1]:
import torch
from models import black_box
torch.manual_seed(42)  # fix random seed

# Set parameters limit（P, v, t, h）
bounds = torch.tensor([
        [25, 0.1, 25],  # Lower bounds
        [300, 0.6, 300]  # Upper bounds
    ], dtype=torch.double)

N_init = 12
X_train = torch.rand(N_init, bounds.shape[1]) * (bounds[1] - bounds[0]) + bounds[0]
Y_train = black_box.mechanical_model(X_train)
print(Y_train)

tensor([[3.0000e+00, 8.0000e+00, 1.6362e+03, 2.8163e+01, 4.2049e+00, 6.3136e-01],
        [2.0000e+00, 1.0000e+01, 1.6825e+03, 6.3829e+01, 3.4945e+00, 6.1373e-01],
        [9.0000e+00, 9.0000e+00, 1.6692e+03, 4.8546e+01, 1.9493e+00, 4.4381e-01],
        [6.0000e+00, 9.0000e+00, 1.5061e+03, 7.3400e+01, 3.8473e+00, 4.4475e-01],
        [4.0000e+00, 9.0000e+00, 1.7424e+03, 5.3169e+01, 1.7920e+00, 4.5420e-01],
        [1.0000e+01, 7.0000e+00, 1.7357e+03, 5.8470e+01, 3.9446e+00, 4.7343e-01],
        [8.0000e+00, 1.0000e+01, 1.5588e+03, 7.6901e+01, 3.4227e+00, 1.0157e+00],
        [1.0000e+01, 1.0000e+01, 1.8273e+03, 5.5323e+01, 2.2536e+00, 2.0930e-01],
        [6.0000e+00, 9.0000e+00, 1.5076e+03, 6.3226e+01, 4.4532e+00, 6.2952e-01],
        [6.0000e+00, 1.0000e+01, 1.4978e+03, 6.0706e+01, 2.0167e+00, 9.1382e-01],
        [3.0000e+00, 1.0000e+01, 1.7700e+03, 4.4432e+01, 1.6612e+00, 5.6012e-01],
        [9.0000e+00, 1.0000e+01, 2.1037e+03, 6.4040e+01, 2.0515e+00, 5.4080e-01]],
       dtype=to

In [7]:
import torch
from botorch.models import SingleTaskGP, ModelListGP
from botorch.models.transforms import Normalize, Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll

# INITIALIZATION
# Sample Specifications: X_k = [density, roughness, time]
K = 8 # Number of Samples
M = 4 # Number of Tasks
N = 3 # Number of Targets

# 假设 train_X 形状为 (N, 4) -> 4个工艺参数
train_X = torch.tensor([
    [100, 800, 0.1, 0.08],
    [120, 1000, 0.1, 0.10],
    [80,  900, 0.12, 0.07],
    [110, 1100, 0.1, 0.09],
    [95,  850, 0.11, 0.08],
    [105, 950, 0.1, 0.07],
    [90,  1000, 0.13, 0.1],
    [115, 970, 0.09, 0.08],
], dtype=torch.double)

# train_Y shape = (N, 3) -> 三个目标
train_Y = torch.tensor([
    [0.98, 8.5, 12.0],   # [Density, Roughness, Time]
    [0.96, 9.0, 10.5],
    [0.92, 10.5, 11.5],
    [0.97, 8.0, 10.0],
    [0.95, 9.3, 12.2],
    [0.96, 9.1, 11.0],
    [0.93, 10.0, 12.5],
    [0.97, 8.7, 10.3],
], dtype=torch.double)

# Single GP，add Normalize and Standardize
model_density = SingleTaskGP(train_X, train_Y[:, 0:1],
                             input_transform=Normalize(d=M),
                             outcome_transform=Standardize(m=1))

model_roughness = SingleTaskGP(train_X, train_Y[:, 1:2],
                               input_transform=Normalize(d=M),
                               outcome_transform=Standardize(m=1))

model_time = SingleTaskGP(train_X, train_Y[:, 2:2+1],
                          input_transform=Normalize(d=M),
                          outcome_transform=Standardize(m=1))
# Merge Model
model = ModelListGP(model_density, model_roughness, model_time)

mlls = [ExactMarginalLogLikelihood(m.likelihood, m) for m in model.models]
for mll in mlls:
    fit_gpytorch_mll(mll)

# TEST
test_X = torch.tensor([
    [100, 950, 0.1, 0.08],
    [110, 1000, 0.1, 0.09],
    [90, 900, 0.12, 0.07],
    [100, 800, 0.1, 0.08],
], dtype=torch.double)

model.eval()
with torch.no_grad():
    posterior_density = model.models[0].posterior(test_X)
    posterior_roughness = model.models[1].posterior(test_X)
    posterior_time = model.models[2].posterior(test_X)

    mean_density = posterior_density.mean
    mean_roughness = posterior_roughness.mean
    mean_time = posterior_time.mean

print("Density prediction:\n", mean_density)
print("Roughness prediction:\n", mean_roughness)
print("Processing Time prediction:\n", mean_time)



Density prediction:
 tensor([[0.9594],
        [0.9601],
        [0.9217],
        [0.9798]], dtype=torch.float64)
Roughness prediction:
 tensor([[ 9.2566],
        [ 8.9068],
        [10.2445],
        [ 8.5048]], dtype=torch.float64)
Processing Time prediction:
 tensor([[11.5986],
        [10.3295],
        [12.1438],
        [11.9870]], dtype=torch.float64)


In [5]:
import torch
test_X = torch.tensor([
    [100, 950, 0.1, 0.08],
    [110, 1000, 0.1, 0.09],
    [90, 900, 0.12, 0.07],
], dtype=torch.double)
print(test_X.shape[4])

3
